# Beat the Offensive Line: Using Data To Determine Blitz Strategy    

Coaching Presentation Track  
Dom Borsani

>*Thank you to Coach Doug Colman for acting as my consultant for this analysis.  He has spent his career coaching football in college and the NFL, and graciously answered my questions about NFL pass protections, defensive fronts, and blitz paths.*  

>*Thank you to Coach Vince Kehres for giving me an opportunity to learn football on his defensive staff at the University of Toledo this season.  I am very grateful for his continued guidance and mentorship.*  

# Introduction

During my time studying and coaching defensive football, I've heard plenty of opinions on how to generate pressure on the quarterback.  Interestingly, I've noticed one common theme: *blitz second level defenders opposite the turn of the center to get 1 on 1 matchups with running backs*.  

Because blitzing requires deception, offensive coaches aim to discover tendencies in the opponent's blitz game, allowing them to adjust their pass protection and negate the blitz.  Therefore, defensive coaches routinely self-scout their own tendencies, to maintain a stealthy blitz attack.  

This analysis presents two predictive models to help defensive coaches during the game plan process:  

  **1) Center Turn Prediction Model for Opponent Scouting**  
  **2) Blitz Prediction Model for Self-Scouting**  

# Football Terminology

### Half-Slide Protection

Blitzing opposite the turn of the center is effective due to the prevalence of half-slide protection.  In half-slide protection, "Half" the line is responsible for a "zone", and "half" the line is responsible for blocking a specific "man".  


<figure>  
    
<img 
     src='https://github.com/dombor100/BDB23/blob/main/half%20slide%20pro.png?raw=true' 
     style="border: 1px solid black"
     width="80%"
     />  
  
</figure>


The left side of the line is the zone side, and the right side of the line is the man side.  The running back (red circle) inserts in between the "zone" side and the "man" side, and is responsible for the linebacker blitz.  So, in the above image, the center turn is to the LEFT. 

### Line of Scrimmage (LOS) and Wide Box

Throughout this writing, if I say someone is "on the LOS", they are positioned within 2 yards of the LOS (depth) and within 5 yards of the tackles / tight ends (width).  Similarly, if a player is in the "wide box", they are within 6 yards of the LOS (depth) and within 5 yards of the tackles / tight ends (width).  

<figure>  
    
<img src='https://github.com/dombor100/BDB23/blob/main/widebox_LOS_description.png?raw=true' style="border: 1px solid black" width="80%"/>  
    
</figure>


# Data Preparation

### Data Cleansing

Data used in this analysis consisted of Next Gen Stats player tracking, play, game, and player information data, as well as corresponding PFF Scouting data for pass plays in weeks 1-8 of the 2021 season.  The data provided were already filtered to remove run, RPO, and screen plays.  However, I also used the following constraints:  

- Play Length >= 3 seconds
- Pass Protection Player Quantity = 5 or 6 (6 man pass protection plays with the TE were NOT included)  
- Offensive Linemen Quantity = 5 (LT, LG, C, RG, RT)  
- TE's in Offensive Formation = 0 or 1  
- RB's in Offensive Formation = 0 or 1  
- TE or offensive linemen does not cross the center after the snap (this removes most bootlegs, nakeds, power passes, etc.)  
- No sprintout plays  
- No QB under center plays (I used plays designated as "shotgun" only)  
- No plays where the center set vertical (vertical set = center does not commit to left or right until a blitzer shows)   

I wanted to focus on dropback pass situations in "open" looking formations, which likely equates to high probability pass situations.  Future work could extend this basis, and/or attempt to predict pass protection schemes in other situations.  

### Feature Engineering & Feature Selection  

*A comprehensive list of features created for each model is located in the [Appendix](#Appendix).*  

The intended use of this analysis is in the defensive game planning realm.  Thus, creating features that are specifically controllable by the defense ensure actionable findings.  

For example, the density plot below shows defensive line location for all 5 man fronts that meet my filters.  The location of an A gap defensive lineman (1/2i technique) influences the turn of the center.  Thus, I incorporated a feature that locates the A gap defensive lineman.  

<figure> 
    
<img src='https://github.com/dombor100/BDB23/blob/main/5man_front_density.PNG?raw=true' style="border: 1px solid black" width="100%"/>  

</figure>  

### Predict Versus Influence  

In general, manipulating the defensive structure allows defensive coaches to *influence* the turn of the center.  On the other hand, certain offensive alignments provide an opportunity to *predict* the turn of the center.  For example, if the offense lines up with three receivers to the right, one receiver to the left, and the running back on the quarterback's left, the center may have a tendency to turn towards the RIGHT, because he typically blocks TOWARD the pass strength and AWAY from the running back.  This would be a *prediction*, because it is dependant on the offense.  However, if the defense overloads the weakside of the offensive line (away from the pass strength), it may *influence* the center to change the protection, and turn LEFT to account for the potential blitzers.  The center turn model helps coaches understand how to *predict* and/or *influence* the turn of the center, week by week.  





# Results & Game Plan Example  

### Set the Scene  

Let's pretend we are defensive analysts for an NFL team.  Each week of the season, we are tasked with presenting recommendations on how to get pressure on the quarterback in true dropback pass situations.  

I've decided we are working for the Dallas Cowboys, because it is Micah Parson's rookie season (2021), and we want to work with versatile players that both can blitz and play coverage.  Parsons tops the list of versatile players:  
- Micah Parsons: 43% Pass Rush - 23 hits, hurries, and sacks  
- Kyle Van Noy: 59% Pass Rush - 8 hits, hurries, and sacks  
- Dont'a Hightower: 52% Pass Rush - 7 hits, hurries, and sacks  

It is week 6, and we play the Patriots.  

### Model Creation  

I used weeks 1-5 for training data and weeks 6-8 for test data.  (I realize if it were REALLY week 6 of the 2021 season, I would not have weeks 6-8 to test the models, but this is just for fun.)  

For both the center turn models and the blitz models, I used the caret package in R to create random forest models for each team.  The center turn models tended to overfit, so I ultimately held mtry constant at 2.  Oppositely, the blitz models performed better with more complexity, so I specified a small search grid for mtry (mtry = 2, 5, or 10).  Since these were random forest-based models, I used the out-of-bag method as the train control, in lieu of bootstrapping or k-fold cross validation (I tried them all, and the performance was very similar for all three).  

### Model Results  

The learning curves for the blitz models show good tradeoff between bias and variance.  

<figure> 
    
<img src='https://github.com/dombor100/BDB23/blob/main/DAL_blitzModel_learningCurves.PNG?raw=true' style="border: 1px solid black" width="100%"/>  

</figure>  

The learning curves for the Patriots center turn models don't look as aesthetically pleasing as the blitz models, but are still acceptable.  In practice, maybe we would incorporate 2020 data to see if it improves our performance.  

<figure>

<img src='https://github.com/dombor100/BDB23/blob/main/NE_centerTurnModel_learningCurves.PNG?raw=true' style="border: 1px solid black" width="100%"/>  

</figure>  
    
    
Here is an example of the models in action.  You can see blitz probabilities for each defender, the gap each wide box player is lined up in, and the center turn probability.  The animation pauses at the snap to display actual defender blitz paths and actual offensive linemen blocking direction.  

<figure>

<img src='https://github.com/dombor100/BDB23/blob/main/animation_1.gif?raw=true' style="border: 1px solid black" width="100%"/>  

</figure>  

### Game Plan Implementation  

##### Center Turn - Variable Importance

Using variable importance calculations, we gain insights into the Patriots center turn tendencies.  

- Center turn = 100% (10/10) RIGHT, when the ball is on the left hash.    
- Center turn = 91% (21/23) RIGHT, when there are more wide box players on the right side.  
- Center turn = 86% (12/14) RIGHT, when there are more "inside" LOS players on the right side.  
- Center turn = 86% (12/14) RIGHT, when there are more "outside" LOS players on the right side.  
- Center turn = 83% (20/24) RIGHT, when the pass strength is to the right.  
- Center turn = 76% (38/50) RIGHT, when there is one running back on the left.  

So, we see that the Patriots have some tendencies based on the way they line up (hash, pass strength, RB location), in addition to tendencies based on how the *defense* lines up (wide box players, LOS players).  Therefore, it is likely that we can *predict* and *influence* the center turn, depending on the situation.  Also, notice all findings are to the RIGHT - this will be important later.  

##### Micah Parsons - Blitz Probability

Using our blitz models, we filter by Micah Parsons' 10 lowest blitz probability plays in weeks 1-5, in which he generated a hit, hurry, or sack.  The top finding shows Parsons (yellow halo) lined up in the wide box, blitzing B gap, opposite the turn of the center.  

>*Parsons' blitz probability at the snap is 79%, which is 17% lower than his average training data blitz probability, when he is blitzing.  So, in this case, 79% is indeed low.  This would NOT be considered low for test data, but remember, training data is always more optimistic than real life.*  

<figure>

<img src='https://github.com/dombor100/BDB23/blob/main/animation_2.gif?raw=true' style="border: 1px solid black" width="100%"/>  

</figure>  


##### Recommendation  

Summarizing the above findings:  

- Variable importance calculations show we can influence the center to turn RIGHT with an LOS overload (see the image below: linebacker "B" is walked up on the LOS to create the overload).  
- Based on the blitz probability models, Parsons could surprise the offense when blitzing the B gap from the second level.    
- The Cowboys ran a similar pressure in week 4, so we know its within their scheme.  

Our recommendation to Coach Quinn is simple and actionable.  The image shows Micah Parsons (11) blitzing the left B gap, with a center turn prediction of 78% right.  

<figure>

<img src='https://github.com/dombor100/BDB23/blob/main/recommendation_gameplan.png?raw=true' style="border: 1px solid black" width="100%"/>  

</figure>


Interestingly, the Cowboys DID run a similar blitz versus the Patriots, but from the opposite side.  

<figure>

<img src='https://github.com/dombor100/BDB23/blob/main/animation_3.gif?raw=true' style="border: 1px solid black" width="100%"/>  

</figure>  

There are a few important points here:  

- Parsons blitzed opposite the turn of the center, with a low blitz probability, generating pressure on the QB.  This is exactly what we wanted.  
- Center turn prediction was 57% left.  The variable importance findings only display center turns to the RIGHT.  This could mean influencing the Patriots center turn to the LEFT is difficult.  
- Unfortunately, the pass was completed for a big gain (sometimes the QB makes a great play).  

It is interesting to see our analysis come to a similar conclusion as Dan Quinn in 2021.  The only difference: we would have flipped the pressure, due to center turn predictability.  

# Conclusions

Center turn prediction models and blitz prediction models were developed to assist defensive coaches in the game planning process.  A use-case example is given, showing the ability to draw actionable conclusions.   

# Appendix  

Code is available [here](https://github.com/dombor100/BDB23).

Word Count = 1774   
Table/Figure Count = 9  

### Complete List of Features

Features used/created for the blitz model, only:  

- **role**: Target variable (from PFF Scouting data).  
- **nflId**: from Player Information data & Player Tracking data.  
- **velocity**: identical to the "speed" variable, but is *negative* if the player is moving away from the LOS.  
- **adjDistanceFromLOS**: the defender's distance from the LOS, adjusted for distance from the ball.  See [explanation](https://github.com/dombor100/BDB23/blob/main/adjDistanceFromLOS%20Expl.png?raw=true).   
- **defenderGap** (factor): gap location of each defenders.  
- **inWideBox_inside_left** (1/0): Any player in the left wide box, INSIDE the tackle or tight end.  
- **inWideBox_outside_left** (1/0): Any player in the left wide box, OUTSIDE the tackle or tight end.  
- **inWideBox_inside_right** (1/0): Any player in the right wide box, INSIDE the tackle or tight end.  
- **inWideBox_outside_right** (1/0): Any player in the right wide box, OUTSIDE the tackle or tight end.  
- **onLOS_inside_left** (1/0): Any player on the left LOS, INSIDE the tackle or tight end.  
- **onLOS_outside_left** (1/0): Any player on the left LOS, OUTSIDE the tackle or tight end.  
- **onLOS_inside_right** (1/0): Any player on the right LOS, INSIDE the tackle or tight end.  
- **onLOS_outside_right** (1/0): Any player on the right LOS, OUTSIDE the tackle or tight end.  

Features created for the center turn model, only:  

- **centerTurn** (left/right): Target variable.  
- **onLOS_Agap_right** (1/0): Any player on LOS, in right A gap.  
- **onLOS_Agap_left** (1/0): Any player on LOS, in left A gap.  
- **onLOS_Agap_right** (1/0): Any player on LOS, in right A gap.  
- **hash** (factor): The location of the ball between the hashes (left, left middle, center, right middle, right).  
- **rightBoxOverload** (1/0): Indicates more box defenders than offensive players on the right.
- **leftBoxOverload** (1/0): Indicates more box defenders than offensive players on the left.
- **rightLOSOverload** (1/0): Indicates more LOS defenders than offensive players on the right.
- **leftLOSOverload** (1/0): Indicates more LOS defenders than offensive players on the left.  
- **inWideBox_LminusR**: Defenders in left wide box MINUS defenders in right wide box.    
- **onLOS_inside_LminusR**: Similar to "onLOS_LminusR"; only includes players INSIDE the tackle or tight end.  
- **onLOS_outside_LminusR**: Similar to "onLOS_LminusR"; only includes players OUTSIDE the tackle or tight end.  
- **secondLevel_LminusR**: Players in the left wide box & NOT on the LOS MINUS players in the right wide box & NOT on the LOS.  
- **inside_2ndLevel_LminusR**: similar to "secondLevel_LminusR"; only includes players INSIDE the tackle or tight end.  
- **outside_2ndLevel_LminusR**: similar to "secondLevel_LminusR"; only includes players OUTSIDE the tackle or tight end.  

Features created for both the center turn prediction and blitz prediction models:  

- **passStrength** (factor): left, right, or balanced.  
- **TEs_right**: TE quantity on right side.    
- **TEs_left**: TE quantity on left side.  
- **RBs_right**: RB quantity on right side.  
- **RBs_left**: RB quantity on left side.  
- **timeToSnap**: time (in seconds) until the ball is snapped.  Obviously, defensive players do not know this explicitly, but I wanted to account for the "feel" of when the offense was ready to snap the ball.  